## Import libraries

In [1]:
# Import lybraries
import numpy as np #numerical computation package
import scipy as sp #library of scientific algorithms
# from scipy.integrate import odeint #to integrate ODEs
# from scipy.spatial.distance import pdist, squareform #to compute Recurrence Plot
# from scipy import signal #to decimate data
import pandas as pd #library to manipulate and arrange data properly
import sympy as sym #symbolic library
import time
from numba import jit #library to compile some functions
import numba #import this way to avoid problems with parallel computation...
import matplotlib.pyplot as plt #library to plot graphs
from mpl_toolkits.mplot3d import Axes3D #plot in 3D
%matplotlib inline
plt.rc('text', usetex=True) #to use Tex fonts in figures
import os.path #to verify existence of a file

In [2]:
# Import created functions
from LF_dynaSysLib import *
# from importlib import reload
# reload(LF_dynaSysLib)

## Oscillators (and their Poincaré sections)
#### Classics

In [3]:
# Rossler: Poincaré section ( x == (c - sqrt(c**2-4*a*b))/2 ) and (dx/dt > 0): f_rossler_poin(x1, x2,...)
def f_rossler_poin(x1, x2, a = .398, b = 2, c = 4):
    """"
    Poincare section proposed by Letellier, C., Dutertre, P., & Maheu, B. (1995). 10.1063/1.166076.
    
    Inputs:
        x1: state of the system to be verified if it is BEFORE the Poincare section
        x2: state of the system to be verified if it is AFTER the Poincare section
        x1.shape = x2.shape = (N, n=3), N: #of points, n=3: order of the system
    
    Default parameters:
       - a = .398, b = 2, c = 4
       - a = 0.165, b = 0.2, c = 10 (alternative)
    
    IFMG - v001 - LFA dez-2020: vectorized implementation
    """
    x1 = x1.reshape(-1, 3) # "3" represents the order of the system
    x2 = x2.reshape(-1, 3)
    xPoin = .5*(c-(c**2-4*a*b)**(.5))
    return (  ( ((x1[:,0]-xPoin)*(x2[:,0]-xPoin))<0 ) & (x2[:,0] > x1[:,0])  )

In [4]:
# Rössler oscillator: f_rossler(x, t, omg = 1, a = 0.165, b = 0.2, c = 10) 
@jit(nopython=True)
def f_rossler(x, t, omg = 1, a = 0.165, b = 0.2, c = 10):
    """
    Rössler oscillator (Rössler, 1976) - Phys. Lett. A, 57(5), 397–398.
        - x[0], x[1], x[2]: state variables sys.(1)  {dx = F(x)
        - t: time
    
    MACSIN - v000 - LFA jul-2017
    """   
    
    # ---------------------------------------------------
    # state vectors
    x1 = x[0]
    y1 = x[1]
    z1 = x[2]
    
    # ---------------------------------------------------
    # Rössler oscillator
    # omg = 1; a = 0.165; b = 0.2; c = 10; #parameters (1)
    return np.array([ -omg*y1-z1 , #drift vector field
                      omg*x1+a*y1 ,
                      b+z1*(x1-c) ])

In [5]:
# Two coupled Rossler systems: f_coupRossler(x, t, eps) 
@jit(nopython=True)
def f_coupRossler(x, t, eps):
    """
    Two coupled hyperchaotic Rossler oscillators:
        - x[0], x[1], x[2]: state variables sys.(1), Rossler
        - x[3], x[4], x[5]: state variables sys.(2), Rossler
        - t: time
        - eps: coupling strength
        - coupVar: coupling variable shape=(3,)
        
    IFMG - v000 - LFA dez-2020
    """   
    # state vectors and fixed parameters
    x1, y1, z1, x2, y2, z2 = x
    
    # SYSTEM (1)
    omg = .98; a = .398; b = 2; c = 4
    F1 = np.array([ -omg*y1-z1 , #drift vector field
                     omg*x1+a*y1 ,
                     b+z1*(x1-c)
    ])
    G1 = np.array([ eps[0]*(x2-x1) , #control vector field
                    eps[1]*(y2-y1) ,
                    eps[2]*(z2-z1)
    ])
    
    # SYSTEM (2)
    omg = 1.02; a = .398; b = 2; c = 4
    F2 = np.array([ -omg*y2-z2 , #drift vector field
                     omg*x2+a*y2 ,
                     b+z2*(x2-c)
    ])
    G2 = np.array([ eps[0]*(x1-x2) , #control vector field
                    eps[1]*(y1-y2) ,
                    eps[2]*(z1-z2)
    ])
    
    # Whole differential equations
    dx = np.hstack((F1 + G1, F2 + G2))
    
    return dx

In [6]:
# Two coupled Hyperchaotic Rossler systems: f_hyperChaosRossler_sync(x, t, eps) 
@numba.jit(nopython=True)
def f_hyperChaosRossler_sync(x, t, eps):
    """
    Two coupled hyperchaotic Rossler oscillators:
        - x[0], x[1], x[2], x[3]: state variables sys.(1), hyperchaotic Rossler
        - x[4], x[5], x[6], x[7]: state variables sys.(2), hyperchaotic Rossler
        - t: time
        - eps: coupling strength
        
        Initial conditions can be: x0 = −10; y0 = −6; z0 = 0; w0 = 10.0;
        
    MACSIN - v000 - LFA abr-2017
    """   
    # state vectors and fixed parameters
    x1, y1, z1, w1, x2, y2, z2, w2 = x
    
    # SYSTEM (1)
    a=0.25; b=3; c=0.5; d=0.05;
    F1 = np.array([ -y1 - z1 , #drift vector field
                    x1 + a*y1 + w1 ,
                    b + x1*z1 ,
                    -c*z1 + d*w1
    ])
    G1 = np.array([ (x2-x1) , #control vector field
                    (y2-y1) ,
                    (z2-z1) ,
                    (w2-w1)
    ])
    
    # SYSTEM (2)
    a=0.255; b=3; c=0.5; d=0.05;
    F2 = np.array([ -y2 - z2 , #drift vector field
                    x2 + a*y2 + w2 ,
                    b + x2*z2 ,
                    -c*z2 + d*w2
    ])
    G2 = np.array([ (x1-x2) , #control vector field
                    (y1-y2) ,
                    (z1-z2) ,
                    (w1-w2)
    ])
    
    # Whole differential equations
    dx = np.hstack((F1,F2)) + np.hstack((eps*G1,eps*G2))
    
    return dx

#### Proposed (old works)

In [7]:
# Coherent Funnel Attractor (modified Rössler): f_coherentFunnel(x, t, omg = 1, a = .3, b = .1, c = 8.5) 
@jit(nopython=True)
def f_coherentFunnel(x, t, omg = 1, a = .3, b = .1, c = 8.5):
    """
    Modified Rössler oscillator
        - x[0], x[1], x[2]: state variables sys.(1)  {dx = F(x)
        - t: time
    
    MACSIN - v000 - LFA jul-2017
    Phase coherence is not related to topology. PHYSICAL REVIEW E, v. 101, p. 032207, 2020.
    http://dx.doi.org/10.1103/PhysRevE.101.032207
    """   
    # state vectors
    x1 = x[0]
    y1 = x[1]
    z1 = x[2]
    
    # differential equation
    fx = np.array([ -omg*y1-z1 , #drift vector field
                     omg*x1+a*y1 ,
                     b+z1*(x1-c) ])
    
    # reparameterization of time
    hx = .546-.471*np.tanh((z1-25.5)/11)
    fx = hx*fx
    
    return fx

In [8]:
# Noncoherent Spiral Attractor (modified Rössler): f_noncoherentSpiral(x, t, omg = 1, a = .16, b = .1, c = 8.5) 
@jit(nopython=True)
def f_noncoherentSpiral(x, t, omg = 1, a = .16, b = .1, c = 8.5):
    """
    Modified Rössler oscillator
        - x[0], x[1], x[2]: state variables sys.(1)  {dx = F(x)
        - t: time
    
    MACSIN - v000 - LFA jul-2017
    Phase coherence is not related to topology. PHYSICAL REVIEW E, v. 101, p. 032207, 2020.
    http://dx.doi.org/10.1103/PhysRevE.101.032207
    """   
    # state vectors
    x1 = x[0]
    y1 = x[1]
    z1 = x[2]
    
    # differential equation
    fx = np.array([ -omg*y1-z1 , #drift vector field
                     omg*x1+a*y1 ,
                     b+z1*(x1-c) ])
    
    # reparameterization of time
    # hx = 21+25*np.tanh((z1-11)/10)
    # hx = 100.1+100*np.tanh((z1-21.5)/7)
    hx = 0.01 + 0.05*(x1**2 + y1**2)
    fx = hx*fx
    
    return fx

#### New Proposal

In [9]:
# Rossler-Z (modified Rössler): f_rosslerZ(x, t, omg = 1, a = .3, b = .1, c = 8.5) 
@jit(nopython=True)
def f_rosslerZ(x, t, omg = 1, a = .398, b = 2, c = 4):
    """
    Modified Rössler oscillator
        - x[0], x[1], x[2]: state variables sys.(1)  {dx = F(x)
        - t: time
        
        Spiral: omg = 1, a = .16, b = .1, c = 8.5
        Funnel: omg = 1, a = .3, b = .1, c = 8.5
        Other: omg = 1, a = .398, b = 2, c = 4 (lifted spiral attractor)
    IFMG - v000 - LFA dez-2020
    """   
    # state vectors
    x1 = x[0]
    y1 = x[1]
    z1 = x[2]
    
    # differential equation
    fx = np.array([ -omg*y1-z1 , #drift vector field
                     omg*x1+a*y1 ,
                     b+z1*(x1-c) ])
    
    # reparameterization of time
    # hx = .546-.471*np.tanh((z1-25.5)/11) #<== from previous paper http://dx.doi.org/10.1103/PhysRevE.101.032207
    # hx = .546-.471*np.tanh((z1-1)/1)
    # hx = 5.546-5.471*np.tanh((z1-1)/1)

    hx = 5.5-5.4*np.tanh((z1-1)/1)
    fx = hx*fx
    
    return fx

In [10]:
# Two coupled Rossler-Z systems: f_coupRosslerZ(x, t, eps) 
@jit(nopython=True)
def f_coupRosslerZ(x, t, eps):
    """
    Two coupled hyperchaotic Rossler oscillators:
        - x[0], x[1], x[2]: state variables sys.(1), Rossler-Z (modified Rossler)
        - x[3], x[4], x[5]: state variables sys.(2), Rossler-Z (modified Rossler)
        - t: time
        - eps: coupling strength
        
    MACSIN - v000 - LFA abr-2017
    """   
    # state vectors and fixed parameters
    x1, y1, z1, x2, y2, z2 = x
    
    # SYSTEM (1)
    omg = 1; a = .398; b = 2; c = 4
    F1 = np.array([ -omg*y1-z1 , #drift vector field
                     omg*x1+a*y1 ,
                     b+z1*(x1-c)
    ])
    G1 = np.array([ eps[0]*(x2-x1) , #control vector field
                    eps[1]*(y2-y1) ,
                    eps[2]*(z2-z1)
    ])
    
    # SYSTEM (2)
    omg = 1; a = .398; b = 2; c = 4
    F2 = np.array([ -omg*y2-z2 , #drift vector field
                     omg*x2+a*y2 ,
                     b+z2*(x2-c)
    ])
    G2 = np.array([ eps[0]*(x1-x2) , #control vector field
                    eps[1]*(y1-y2) ,
                    eps[2]*(z1-z2)
    ])
    
#     # reparameterization of time ####################<=== OLD
#     hx1 = 5.546-5.471*np.tanh((z1-1)/1)
#     F1 = hx1*F1
#     hx2 = 5.546-5.471*np.tanh((z2-1)/1)
#     F2 = hx2*F2
    
    # reparameterization of time
    hx1 = 5.5-5.4*np.tanh((z1-1)/1)
    F1 = hx1*F1
    hx2 = 5.5-5.4*np.tanh((z2-1)/1)
    F2 = hx2*F2
    
    # Whole differential equations
    dx = np.hstack((F1 + G1, F2 + G2))
    
    return dx

## EXPERIMENT CLASSES

Classes used to define an experiment and metrics

In [11]:
# CLASS: syncExperiment, with the appropriate functions 
class syncExperiment:
    def __init__(self, descricao=""):
        """
        Class used to concentrate data of an experiment.
        """
        self.descricao = descricao
        print("Object created.")
    
    ############################################################
    # SIMULATION FUNCTIONS
    def simulate(self, ode, eps, dt, x0tr, n=3, tfTr=500, tf=5000):
        # PARAMETERS
        self.ode = ode #ordinary differential equation
        self.eps = eps #coupling strength
        self.dt = dt #integration step
        self.x0tr = x0tr #initial condition
        self.n = n #order of the oscillator(s)
        self.tfTr = tfTr #duration of the TRANSIENT regime
        self.tf = tf #duration of the PERMANENT regime
        
        # TRANSIENT simulation
        print("Running transient simulation...")
        self.tTr = np.arange(0, self.tfTr, self.dt) #transient
        self.xTr = intRK2(self.ode, self.x0tr, self.tTr, (0*self.eps,)) #simulate the systems without coupling
        self.xTr = intRK2(self.ode, self.xTr[-1], self.tTr, (self.eps,)) #simulate COUPLED systems
        
        # SIMULATION on permanent regime
        print("Running permanent regime simulation...")
        self.T = np.arange(0, self.tf, self.dt) #time range
        self.X = intRK2(self.ode, self.xTr[-1], self.T, (self.eps,)) #use 'intRK' or 'odeint'
        print("End of simulation.")
        
    def computeVFP(self, f_poincare):
        self.poincare = f_poincare #function to compute Poincare section
        
        print("Computing vector field phase (VFP)...")
        # PHASE VARIABLE - compute VFP
        self.timeBetCross_1, self.crossIdx_1, self.numPass_1 = f_timeBetCrossPoin(
            self.X[:,:self.n], self.T, self.poincare)
        self.timeBetCross_2, self.crossIdx_2, self.numPass_2 = f_timeBetCrossPoin(
            self.X[:,self.n:], self.T, self.poincare)
        
        # Phase variable (Poincaré section)
        self.phi_a_1 = np.interp(self.T, self.T[self.crossIdx_1], 2*np.pi*np.arange(len(self.crossIdx_1)))
        self.phi_a_2 = np.interp(self.T, self.T[self.crossIdx_2], 2*np.pi*np.arange(len(self.crossIdx_2)))

        # Phase variable (VFP)
        self.vfp_1, self.vfp_2, self.ell1, self.ell2, self.freq1, self.freq2 = f_estimateVFP(
            self.X, self.T, self.crossIdx_1, self.crossIdx_2, lambda x: self.ode(x=x, t=0, eps=self.eps))
        print("VFP computed.")
    
    ############################################################
    # PERFORMANCE FUNCTIONS
    def syncMetrics(self):
        # VFP based metrics
        phDiff = self.vfp_1 - self.vfp_2
        # Mean Phase Coherence (ref: https://doi.org/10.1016/S0167-2789(00)00087-7)
        self.VFP_Rcoh = np.sqrt( np.mean(np.sin(np.abs(phDiff)))**2 + np.mean(np.cos(np.abs(phDiff)))**2 )
        # Max Absolute phase difference
        self.VFP_maxDiff = np.max(np.abs(phDiff))
        self.VFP_meanDiff = np.mean(phDiff) #mean phase diff
        self.VFP_varDiff = np.var(phDiff) #variance of the phase diff
        
        # Poincaré phase metrics (same metrics above)
        phDiff = self.phi_a_1 - self.phi_a_2
        self.phi_a_Rcoh = np.sqrt( np.mean(np.sin(np.abs(phDiff)))**2 + np.mean(np.cos(np.abs(phDiff)))**2 )
        self.phi_a_maxDiff = np.max(np.abs(phDiff))
        self.phi_a_meanDiff = np.mean(phDiff)
        self.phi_a_varDiff = np.var(phDiff)
    
    ############################################################
    # DATA VISUALIZATION PROCEDURES
    def plotPhaseDiffVFP(self):
        fig = plt.figure(figsize=(6,1), dpi=150)
        plt.title("Phase difference (VFP) $\epsilon_x=%.3f$, $\epsilon_y=%.3f$, $\epsilon_z=%.3f$"%
                  (self.eps[0], self.eps[1], self.eps[2]))
        plt.plot(self.T, self.vfp_1-self.vfp_2)
        plt.ylabel("VFP phase diff")
        plt.xlabel("$t$")
        plt.xlim(min(self.T), max(self.T))
        
    def plotPhaseDiffPoin(self):
        fig = plt.figure(figsize=(6,1), dpi=150)
        plt.title("Phase difference (Poincaré) $\epsilon_x=%.3f$, $\epsilon_y=%.3f$, $\epsilon_z=%.3f$"%
                  (self.eps[0], self.eps[1], self.eps[2]))
        plt.plot(self.T, self.phi_a_1-self.phi_a_2)
        plt.ylabel("Poin phase diff")
        plt.xlabel("$t$")
        plt.xlim(min(self.T), max(self.T))
        

In [21]:
# CLASS: syncExpToSave, where only the main informations are saved (exclude simulated)
class syncExpToSave():
    def __init__(self, exp=None):
        """
        Class used to save an experiment with using low memory.
        The vectors of time, states, phase, are not included.
        """
        if exp is not None:
            self.descricao = exp.descricao
            self.ode = exp.ode
            self.eps = exp.eps
            self.dt = exp.dt
            self.x0tr = exp.x0tr
            self.n = exp.n
            self.tfTr = exp.tfTr
            self.tf = exp.tf
            self.tTr = exp.tTr
            self.xTr = exp.xTr
            
            #self.poincare = exp.poincare
            
            ######################################### TESTE RETIRANDO PARÂMETRO POINCARE
            self.poincare = 0

            self.VFP_Rcoh = exp.VFP_Rcoh
            self.VFP_maxDiff = exp.VFP_maxDiff
            self.VFP_meanDiff = exp.VFP_meanDiff
            self.VFP_varDiff = exp.VFP_varDiff
            self.phi_a_Rcoh = exp.phi_a_Rcoh
            self.phi_a_maxDiff = exp.phi_a_maxDiff
            self.phi_a_meanDiff = exp.phi_a_meanDiff
            self.phi_a_varDiff = exp.phi_a_varDiff
        
        print("syncExpToSave object created.")
    
    def createSyncExp(self):
        """
        Function to creat a "sync experiment" object based on the "syncExpToSave" object.
        """
        # Create "sync experiment" object
        exp = syncExperiment()

        # Run Simulation!
        exp.simulate(ode=self.ode,
                     eps=self.eps,
                     dt=self.dt, x0tr=self.x0tr, n=self.n, tfTr=self.tfTr, tf=self.tf)

        # Compute VFP
        exp.computeVFP(f_poincare=self.poincare)

        # Compute sync metrics
        exp.syncMetrics()
        
        return exp
    
    def saveSyncExp(self, fileName):
        # save data in a defined oder
        data = np.asarray((self.descricao, self.ode, self.eps,
                   self.dt, self.x0tr, self.n, self.tfTr,
                   self.tf, self.tTr, self.xTr, self.poincare,
                   self.VFP_Rcoh, self.VFP_maxDiff, self.VFP_meanDiff,
                   self.VFP_varDiff, self.phi_a_Rcoh, self.phi_a_maxDiff,
                   self.phi_a_meanDiff, self.phi_a_varDiff), dtype=object)
        
        # save data compressed
        np.savez_compressed(fileName+".npz", data)
        return 0
    
    def loadSyncexp(self, fileName):
        # load dict of arrays
        dict_data = np.load(fileName+".npz", allow_pickle=True)
        # restore data in the same order
        self.descricao, self.ode, self.eps, \
        self.dt, self.x0tr, self.n, self.tfTr, \
        self.tf, self.tTr, self.xTr, self.poincare, \
        self.VFP_Rcoh, self.VFP_maxDiff, self.VFP_meanDiff, \
        self.VFP_varDiff, self.phi_a_Rcoh, self.phi_a_maxDiff, \
        self.phi_a_meanDiff, self.phi_a_varDiff = dict_data['arr_0']
        return 0

Set experiment number

### Parallel processing

In [23]:
import joblib as jb
N_workers = jb.cpu_count()
print('- N_workers = ',N_workers)

- N_workers =  8


In [22]:
# Sync Experiment: runExperiment(eps_Z, runNumber=0)
def runExperiment(eps_Z, runNumber=0):
    
    # Seed
    np.random.seed(runNumber)
    
    # Initial condition
    n = 3
    x0tr = 1*np.random.rand(2*n) #random initial condition
    
    # Create "sync experiment" object
    exp1 = syncExperiment()
    
    # Run Simulation!
    exp1.simulate(ode=f_coupRossler,
                  eps=np.array([0.0, 0.0, eps_Z]),
                  dt=.01, x0tr=x0tr, n=n, tfTr=500, tf=5000)
    
    # Compute VFP
    exp1.computeVFP(f_poincare=f_rossler_poin)
    
    # Compute sync metrics
    exp1.syncMetrics()
    
    # Convert to a low memory version
    exp1save = syncExpToSave(exp1)
    
    # Save results
    fileName = 'exp/Rossler_eps_z_%.3d_run_%.4d'%(int(eps_Z*1000), runNumber)
    exp1save.saveSyncExp(fileName)
    
    return runNumber+eps_Z

# Function to execute Monte Carlo runs
def monteCarloRuns(epsZ, totalRuns = 100):
    for runNum in range(totalRuns):
        runExperiment(eps_Z=epsZ, runNumber=runNum)
    return epsZ

In [ ]:
# PARAMETERS of the experiment
# Total number of Monte Carlo runs
totalRuns = 100
# coupling strength values
epsZ_values = np.arange(.01, .5, .01)

tic = time.time()
if __name__ == '__main__':
    res=jb.Parallel(n_jobs=-1, verbose=1)(jb.delayed(monteCarloRuns)
                                          (epsZ, totalRuns)
                                          for epsZ in epsZ_values)
toc=time.time()
elapsed=toc-tic
print('- Elapsed time: %.4f s, %.4f min, %.4f h.'%(elapsed, elapsed/60, elapsed/3600))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


# Tests executed to verify classes

In [144]:
# TEST CLASSES, save, load...

# Initial condition
n = 3
np.random.seed(2021)
x0tr = 1*np.random.rand(2*n) #random initial condition
# Create "sync experiment" object
exp1 = syncExperiment()
# Run Simulation!
exp1.simulate(ode=f_coupRossler,
              eps=np.array([0.0, 0.0, 0.7]),
              dt=.01, x0tr=x0tr, n=n, tfTr=500, tf=5000)
# Compute VFP
exp1.computeVFP(f_poincare=f_rossler_poin)
# Compute sync metrics
exp1.syncMetrics()

# Test ==> CHANGE OBJECT TYPE
exp1save = syncExpToSave(exp1)
print(exp1save.VFP_varDiff == exp1.VFP_varDiff)
# Test ==> SAVE FILE
exp1save.saveSyncExp(fileName="exp/teste123")
exp1loaded = syncExpToSave() #create empty object
# Test ==> LOAD FILE
exp1loaded.loadSyncexp(fileName="exp/teste123")
a_val = [value for value, _ in exp1save.__dict__.items()]
b_val = [value for value, _ in exp1loaded.__dict__.items()]
# Test if all attribute values are equal
for k in range(19):
    print(a_val[k] == b_val[k])
a_att = [att for _, att in exp1save.__dict__.items()]
b_att = [att for _, att in exp1loaded.__dict__.items()]
# Test if all attribute names are equal
for k in range(19):
    print(a_att[k] == b_att[k])
########### ALL TESTS ARE OK! ==> 07-02-2021

Object created.
Running transient simulation...
Running permanent regime simulation...
End of simulation.
Computing vector field phase (VFP)...
VFP computed.
syncExpToSave object created.
True
syncExpToSave object created.
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
[ True  True  True]
True
[ True  True  True  True  True  True]
True
True
True
[ True  True  True ...  True  True  True]
[[ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 ...
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]]
True
True
True
True
True
True
True
True
True


In [17]:
# Testing MONTE CARLO RUN functions ==> ok ~9.48 seconds
tic = time.time()
runExperiment(eps_Z=.012, runNumber=123)
tac = time.time()
print(" ")
print(tac-tic)
# Test ==> ok!
monteCarloRuns(.03, 2)

Object created.
Running transient simulation...
Running permanent regime simulation...
End of simulation.
Computing vector field phase (VFP)...
VFP computed.
syncExpToSave object created.
 
8.827789306640625
Object created.
Running transient simulation...
Running permanent regime simulation...
End of simulation.
Computing vector field phase (VFP)...
VFP computed.
syncExpToSave object created.
Object created.
Running transient simulation...
Running permanent regime simulation...
End of simulation.
Computing vector field phase (VFP)...
VFP computed.
syncExpToSave object created.


0.03